In [1]:
from glob import glob
import pandas as pd

In [2]:
columns = ['price', 'quantity', 'timestamp', 'is_buyer_maker']

spot_df = []
derivative_df = []

for file in glob('data/*'):
    if 'spot' in file:
        spot_df.append(pd.read_csv(file, header=None))
    else:
        derivative_df.append(pd.read_csv(file, header=None)[1:])

spot_df = pd.concat(spot_df)[[1, 2, 5, 6]].reset_index(drop=True)
derivative_df = pd.concat(derivative_df)[[1, 2, 5, 6]].reset_index(drop=True)

spot_df.columns = columns
derivative_df.columns = columns



spot_df['timestamp'] = pd.to_datetime(spot_df['timestamp'], unit='ms')
derivative_df['timestamp'] = pd.to_datetime(derivative_df['timestamp'], unit='ms')


spot_df = spot_df.sort_values('timestamp').reset_index(drop=True)
derivative_df = derivative_df.sort_values('timestamp').reset_index(drop=True)


derivative_df['derivative_price'] = derivative_df['price']


/tmp/ipykernel_83025/1821862887.py:10: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  derivative_df.append(pd.read_csv(file, header=None)[1:])
/tmp/ipykernel_83025/1821862887.py:10: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  derivative_df.append(pd.read_csv(file, header=None)[1:])
/tmp/ipykernel_83025/1821862887.py:10: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  derivative_df.append(pd.read_csv(file, header=None)[1:])


In [3]:
df = pd.merge_asof(spot_df[['timestamp', 'price', 'quantity', 'is_buyer_maker']], derivative_df[['timestamp', 'derivative_price']], on='timestamp')

In [4]:
len(df)

20509075

In [5]:
params = {}
params['capital'] = 20000
params['maxPositionSize'] = 10000

curr_pos = 0
hedge_pos = 0

buys = 0
sells = 0
buy_volume = 0
sell_volume = 0

deets = []


for idx, row in df.iterrows():
    curr_deets = {}

    if (idx % 1000) == 0:
        curr_deets['timestamp'] = row['timestamp']
        curr_deets['curr_pos'] = curr_pos * row['price']
        curr_deets['capital'] = params['capital']
        curr_deets['worth'] = curr_deets['curr_pos'] + curr_deets['capital']
        curr_deets['price'] = row['price']
        deets.append(curr_deets)
        
    if row['is_buyer_maker']:
        if (curr_pos * row['price']) < params['maxPositionSize']:

            buy_amt = row['quantity'] * row['price']
            quantity = row['quantity']

            if buy_amt > params['maxPositionSize']:
                buy_amt = params['maxPositionSize']
                quantity = buy_amt / row['price']

            buys = buys + 1
            buy_volume = buy_volume + buy_amt

                # print("BUYING {} BTC for {} USD. Current POS: {}".format(quantity, buy_amt, curr_pos))
                curr_pos = curr_pos + quantity
                params['capital'] = params['capital'] - buy_amt
    else:
        if curr_pos > 0:
            sell_amt = row['quantity'] * row['price']
            quantity = row['quantity']

            if sell_amt > curr_pos:
                sell_amt = curr_pos
                quantity = sell_amt / row['price']

            sells = sells + 1
            sell_volume = sell_volume + sell_amt

            # print("SELLING {} BTC for {} USD. Current POS: {}".format(quantity, sell_amt, curr_pos))
            curr_pos = curr_pos - quantity
            params['capital'] = params['capital'] + sell_amt

In [6]:
print(buys, sells, buy_volume, sell_volume)

8903 10231079 5016696.383712207 5006984.2770207785


In [7]:
(buy_volume + sell_volume)

10023680.660732985

In [8]:
details_df = pd.DataFrame(deets)

In [11]:
import plotly.graph_objects as go

fig = go.Figure(layout=go.Layout(xaxis={'spikemode': 'across'}))
fig.add_trace(go.Scatter(x=details_df['timestamp'], y=details_df['worth'], name='worth', yaxis="y1"))
fig.add_trace(go.Scatter(x=details_df['timestamp'], y=details_df['price'], name='price', yaxis="y2"))

fig.update_layout(
            yaxis=dict(
                titlefont=dict(
                    color="#000000"
                ),
                tickfont=dict(
                    color="#000000"
                )
            ),
            yaxis2=dict(
                anchor="free",
                overlaying="y",
                side="left",
                position=1
            )
            )

In [12]:
open('plot_long.html', 'w').write(fig.to_html())

5335474